In [ ]:
import numpy as np
from scipy.linalg import expm, eig, solve, lstsq
import matplotlib.pyplot as plt

In [ ]:
x = np.array([[0,0.5],[0.5,0]])
y = np.array([[0,-0.5j],[0.5j,0]])
z = np.array([[0.5,0],[0,-0.5]])
sx = np.kron(x, np.identity(2))
sy = np.kron(y, np.identity(2))
sz = np.kron(z, np.identity(2))
ix = np.kron(np.identity(2), x)
iy = np.kron(np.identity(2), y)
iz = np.kron(np.identity(2), z)

In [ ]:
ham_sz = 400e6 * np.kron(z, np.identity(2))
ham_iz = 400e6 * np.kron(np.identity(2), z)
ham_hf = 2e6 * np.kron(z, x)
ham_emr = 1e6 * np.kron(x, np.identity(2))

In [ ]:
ham_offset = 263e9 * np.kron(z, np.identity(2))

In [ ]:
ham_int = ham_sz + ham_iz + ham_hf
ham_eff = ham_int + ham_emr

In [ ]:
ham0 = ham_int + ham_offset

In [ ]:
ham_lab = ham_eff + ham_offset

In [ ]:
h = 6.62607004e-34
k = 1.38064852e-23
T = 100 # temperature

In [ ]:
rho0 = expm(-(h/(k*T)) * ham0)/np.trace(expm(-(h/(k*T)) * ham0))

In [ ]:
rho_eq = expm(-(h/(k*T)) * ham_lab)/np.trace(expm(-(h/(k*T)) * ham_lab))

In [ ]:
rho_eq_super = rho_eq.flatten()[np.newaxis].T

In [ ]:
dt = 1e2/263e9

In [ ]:
D = expm(-2.0j * np.pi * ham_offset * dt)
print(D)

In [ ]:
def commutationSuperOp(op):
    res = np.kron(op, np.identity(op.shape[0])) - np.kron(np.identity(op.shape[0]), op.T)
    return res

In [ ]:
h_super = commutationSuperOp(ham_eff)

In [ ]:
eigenval, eigenvec = eig(ham_lab)

In [ ]:
t1e = 1.0e-3
t2e = 1.0e-6
t1n = 10
t2n = 4.0e-3

In [ ]:
def secularRelaxationSuperOp(op):
    Aq = commutationSuperOp(op)
    Anq = commutationSuperOp(op.conjugate().T)
    return np.dot(Anq, Aq)

In [ ]:
def t1SuperOp(t1, eigenvec, x, y):
    x_rotated = np.dot(eigenvec, np.dot(x, eigenvec.conjugate().T))
    y_rotated = np.dot(eigenvec, np.dot(y, eigenvec.conjugate().T))
    
    part_x = secularRelaxationSuperOp(x_rotated)
    part_y = secularRelaxationSuperOp(y_rotated)
    
    return 0.5 * 1.0/t1 * (part_x + part_y)

In [ ]:
def t2SuperOp(t2, eigenvec, z):
    z_rotated = np.dot(eigenvec, np.dot(z, eigenvec.conjugate().T))
    
    part_z = secularRelaxationSuperOp(z_rotated)
    
    return 1.0/t2 * part_z

In [ ]:
gamma_t1e = t1SuperOp(t1e, eigenvec, sx, sy)
gamma_t1n = t1SuperOp(t1n, eigenvec, ix, iy)
gamma_t2e = t2SuperOp(t2e, eigenvec, sz)
gamma_t2n = t2SuperOp(t2n, eigenvec, iz)

In [ ]:
gamma_lab = gamma_t1e + gamma_t2e + gamma_t1n + gamma_t2n

In [ ]:
L = 1.0j * h_super + gamma_lab

In [ ]:
gamma_rho_eq_super = np.dot(gamma_lab, rho_eq_super)

In [ ]:
results = lstsq(L, gamma_rho_eq_super)

In [ ]:
L

In [ ]:
for i in range(L.shape[0]):
    print('{\n  ', end='')
    for j in range(L.shape[1]-1):
        print('cxdbl({}, {}), '.format(L[i,j].real, L[i,j].imag), end='')
    print('cxdbl({}, {}), '.format(L[i,L.shape[1]-1].real, L[i,L.shape[1]-1].imag), end='')
    print('\n},')

In [ ]:
gamma_rho_eq_super - np.dot(L, results[0])

In [ ]:
for i in range(gamma_rho_eq_super.shape[0]):
    print('{\n  ', end='')
    for j in range(gamma_rho_eq_super.shape[1]-1):
        print('cxdbl({}, {}), '.format(gamma_rho_eq_super[i,j].real, gamma_rho_eq_super[i,j].imag), end='')
    print('cxdbl({}, {}), '.format(gamma_rho_eq_super[i,gamma_rho_eq_super.shape[1]-1].real, 
                                   gamma_rho_eq_super[i,gamma_rho_eq_super.shape[1]-1].imag), end='')
    print('\n},')

In [ ]:
results[0]

In [ ]:
for i in range(results[0].shape[0]):
    print('{\n  ', end='')
    for j in range(results[0].shape[1]-1):
        print('cxdbl({}, {}), '.format(results[0][i,j].real, results[0][i,j].imag), end='')
    print('cxdbl({}, {}), '.format(results[0][i,results[0].shape[1]-1].real, 
                                   results[0][i,results[0].shape[1]-1].imag), end='')
    print('\n},')

In [ ]:
acq_e, acq_h = np.kron(z, np.identity(2)), np.kron(np.identity(2), z)

In [ ]:
acq_e = acq_e.flatten()[np.newaxis]
acq_h = acq_h.flatten()[np.newaxis]

In [ ]:
result = results[0]

In [ ]:
val_e = np.dot(acq_e.conjugate(), result)
val_h = np.dot(acq_h.conjugate(), result)
print(val_e, val_h)